<a href="https://colab.research.google.com/github/nateraw/huggingface-hub-examples/blob/main/speech_to_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! pip install speechbrain huggingface-hub gradio
! pip install git+https://github.com/finetuneanon/transformers@gpt-neo-localattention3-rp-b
! apt install git-lfs

In [ ]:
import gradio as gr
from speechbrain.pretrained import EncoderDecoderASR
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    GPTNeoForCausalLM,
)
from huggingface_hub import snapshot_download

## Load ASR Model

In [ ]:
asr_model = EncoderDecoderASR.from_hparams(
    source="speechbrain/asr-transformer-transformerlm-librispeech",
    savedir="pretrained_models/asr-transformer-transformerlm-librispeech",
    run_opts={"device":"cuda"}
)

## Load Python Code Completion Model

This one takes quite a while to download, so maybe go get a coffee? ☕

In [ ]:
repo_dir = snapshot_download("NovelAI/genji-python-6B-split")
model = AutoModelForCausalLM.from_pretrained(repo_dir + "/model").half().eval().cuda()
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")

## Launch App

In [ ]:
def speech_to_text(file):
    text = asr_model.transcribe_file(file.name)
    text = f"'''{text.lower()}'''\n"

    tokens = tokenizer(text, return_tensors="pt").input_ids
    generated_tokens = model.generate(
        tokens.long().cuda(),
        use_cache=True,
        do_sample=True,
        top_k=50,
        temperature=0.3,
        top_p=0.9,
        repetition_penalty=1.125,
        min_length=1,
        max_length=len(tokens[0]) + 400,
        pad_token_id=tokenizer.eos_token_id
    )
    last_tokens = generated_tokens[0][len(tokens[0]):]
    generated_text = tokenizer.decode(last_tokens)
    return text + generated_text

iface = gr.Interface(
    fn=speech_to_text,
    inputs=gr.inputs.Audio(source="microphone", type="file", label=None),
    outputs="text"
)
iface.launch()